# **Check Data Quality**

## **Import libraries and Load data** ##

#### **Import Required Libraries**

In [1]:
import pandas as pd
import glob
from collections import defaultdict

#### **Load All Parquet Files**
* Reads all `.parquet` files from the specified path, covering all years, months, days and hours.
* If no files are found, it raises a `FileNotFoundError`.
* If files exist, they are concatenated into a single `DataFrame`.

In [2]:
files = glob.glob("/home/jovyan/data/data.parquet/year=*/month=*/day=*/hour=*/*.parquet")

if len(files) == 0:
    raise FileNotFoundError("❌ Can't find parquet in this path")

df = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)

## **Quality Check** ##

#### **Validate Dataset Schema**
- Checks whether the dataset contains all the required columns.  
- If any columns are missing, it raises a `ValueError` with the list of missing columns.  
- If all required columns are present, it prints a verification message.

In [3]:
expected_columns = [
    "timestamp", "stationID", "nameTH", "areaTH", "district",
    "lat", "long", "AQI.aqi", "PM25.value"
]

missing_columns = [col for col in expected_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"❌ Missing columns: {missing_columns}")
else:
    print("\n✅ Schema is correct.")


✅ Schema is correct.


#### **Check Minimum Record Count (≥ 1,000 records)**
- Ensures the dataset has at least 1,000 records.  
- If the record count is lower than expected, it displays a warning message.

In [4]:
if len(df) >= 1000:
    print(f"\n✅ Record count verified: {len(df):,} records.")
else:
    print(f"\n❌ Too few records: {len(df)} (need ≥ 1000)")


✅ Record count verified: 13,440 records.


#### **Check 24-Hour Coverage**
- This step verifies the **completeness of hourly data** for each calendar day in the dataset.
- Converts `timestamp` to datetime, then extracts `year`, `month`, `day`, and `hour`.
- Then, it groups the data by `(year, month, day)` and counts the number of unique `hour` values per day.
- A complete day must contain exactly **24 distinct hourly records** (from 00 to 23).
- If any day has fewer than 24 hours, the script will print the specific date along with the list of missing hour values.
- If all days are complete, a verification message is printed.

In [5]:
df_for_24_hrs = df.copy()
df_for_24_hrs['timestamp'] = pd.to_datetime(df_for_24_hrs['timestamp'])

df_for_24_hrs['year'] = df_for_24_hrs['timestamp'].dt.year
df_for_24_hrs['month'] = df_for_24_hrs['timestamp'].dt.month
df_for_24_hrs['day'] = df_for_24_hrs['timestamp'].dt.day
df_for_24_hrs['hour'] = df_for_24_hrs['timestamp'].dt.hour

grouped = df_for_24_hrs.groupby(['year', 'month', 'day'])['hour'].nunique()
incomplete_days = grouped[grouped < 24]

missing_hours = defaultdict(list)

for (y, m, d), group_df in df_for_24_hrs.groupby(['year', 'month', 'day']):
    hours = set(group_df['hour'])
    expected_hours = set(range(24))
    missing = sorted(expected_hours - hours)
    if missing:
        missing_hours[(y, m, d)] = missing

if not missing_hours:
    print("\n✅ All days include complete 24-hour data records.")
else:
    print(f"\n❌ Found {len(missing_hours)} incomplete day(s):")
    for (y, m, d), hours in missing_hours.items():
        print(f"\n  - {y}-{m:02d}-{d:02d}: missing hours {hours}")


✅ All days include complete 24-hour data records.


#### **Check Column Completeness (≥ 90.00 %)**
- Calculates the percentage of non-null values for each column.  
- If any column has less than 90.00 % completeness, it reports the column name and completeness percentage.  
- If all columns are ≥ 90.00 % complete, a verification message is displayed.

In [6]:
missing_ratio = df.isnull().mean()
completeness_ratio = (1 - missing_ratio) * 100.00

completeness_str = completeness_ratio.apply(lambda x: f"{x:6.2f} %")

print("\n-- Column completeness (%) --")
max_col_len = max(len(col) for col in completeness_str.index)
for col, val in completeness_str.items():
    print(f"\n{col.ljust(max_col_len)} : {val}")

incomplete_cols = completeness_ratio[completeness_ratio < 90.00]

if incomplete_cols.empty:
    print("\n✅ Completeness ≥ 90.00 % for all columns.")
else:
    print("\n❌ Some columns have < 90.00 % completeness:")
    for col, val in incomplete_cols.items():
        print(f"{col.ljust(max_col_len)} : {val:6.2f}%")


-- Column completeness (%) --

timestamp  : 100.00 %

stationID  : 100.00 %

nameTH     : 100.00 %

areaTH     : 100.00 %

district   : 100.00 %

lat        : 100.00 %

long       : 100.00 %

AQI.aqi    : 100.00 %

PM25.value : 100.00 %

✅ Completeness ≥ 90.00 % for all columns.


#### **Check for Object Columns**
- Lists the data type of each column in the dataset.  
- Checks if any columns are of type `object` 
- If no object-type columns are found, a verification message is shown.

In [7]:
dtypes = df.dtypes.astype(str)
max_col_len = max(len(col) for col in dtypes.index)

print("\n-- Column Data Types --")
for col, dtype in dtypes.items():
    print(f"\n{col.ljust(max_col_len)} : {dtype}")

object_cols = dtypes[dtypes == 'object'].index.tolist()

if not object_cols:
    print("\n✅ No object-type columns found.")
else:
    print(f"\n❌ Found object-type columns: {object_cols}")


-- Column Data Types --

timestamp  : datetime64[us]

stationID  : string

nameTH     : string

areaTH     : string

district   : string

lat        : float64

long       : float64

AQI.aqi    : int64

PM25.value : float64

✅ No object-type columns found.


#### **Check for Duplicate Rows**
- Checks for fully duplicated rows in the dataset using `df.duplicated()`.  
- Reports the number of duplicate rows found, or shows a verification message if none are present.

In [8]:
duplicate_count = df.duplicated().sum()
if duplicate_count == 0:
    print("\n✅ No duplicated rows found.")
else:
    print(f"\n❌ Found {duplicate_count} duplicate rows")


✅ No duplicated rows found.


#### **Display DataFrame Summary**
- Uses `df.info()` to display an overview of the dataset.
- For a quick inspection of the data structure.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13440 entries, 0 to 13439
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   13440 non-null  datetime64[us]
 1   stationID   13440 non-null  string        
 2   nameTH      13440 non-null  string        
 3   areaTH      13440 non-null  string        
 4   district    13440 non-null  string        
 5   lat         13440 non-null  float64       
 6   long        13440 non-null  float64       
 7   AQI.aqi     13440 non-null  int64         
 8   PM25.value  13440 non-null  float64       
dtypes: datetime64[us](1), float64(3), int64(1), string(4)
memory usage: 945.1 KB
